
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [41]:

# Part 1
import pandas as pd
import numpy as np
import sklearn as skl
from sklearn.model_selection import train_test_split

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.
path = 'Corona_NLP_train.csv'
df = pd.read_csv(path,encoding="latin-1")
print(df.shape)
# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.
# Make sure that all your data is numpy arrays and not pandas dataframes or series.
cleaned = df[df["Sentiment"] != 'Neutral']
Y= cleaned["Sentiment"]
Y = Y.map({"Positive":1,"Negative":0, "Extremely Negative":0, "Extremely Positive":1})
X = cleaned.drop(columns="Sentiment")["OriginalTweet"]
print(X.shape)
N = len(X)
train_end = int(0.60 * N)     # 60% of data
test_end = train_end + int(0.15 * N)  # next 15% of data

X_train, Y_train = X[:train_end], Y[:train_end]
X_test,  Y_test  = X[train_end:test_end],  Y[train_end:test_end]
X_val,   Y_val   = X[test_end:], Y[test_end:]


(41157, 6)
(33444,)


In [42]:

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

model = Pipeline([
    ("vectorizer", CountVectorizer()), 
    ("classifier", LogisticRegression(max_iter=1000, solver='liblinear'))
])

model.fit(X_train, Y_train)

from sklearn.metrics import accuracy_score, classification_report
Y_pred = model.predict(X_test)
pred_proba = model.predict_proba(X_test)
print("first predicted probabilities: ", pred_proba[0])
# Evaluate
print("precision:", precision_score(Y_test, Y_pred))
print("recall:", recall_score(Y_test, Y_pred))

#print(classification_report(Y_test, Y_pred))


first predicted probabilities:  [0.87757459 0.12242541]
precision: 0.8708971553610503
recall: 0.8808557727775729


In [43]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.

precision_1 = precision_score(Y_test, Y_pred, pos_label=1)
recall_1 = recall_score(Y_test, Y_pred, pos_label=1)

# Negative (label=0)
precision_0 = precision_score(Y_test, Y_pred, pos_label=0)
recall_0 = recall_score(Y_test, Y_pred, pos_label=0)

In [44]:
# Part 4

def cost(model,threshold,X,Y):
    prob_pos = model.predict_proba(X)[:, 1]

    # 2. Predict 1 if prob >= threshold, else 0
    Y_pred = (prob_pos >= threshold).astype(int)

    # 3. Compute the cost
    #    cost = 1 if Y=1 and Y_pred=0
    #          = 5 if Y=0 and Y_pred=1
    #          = 0 otherwise
    cost_array = np.where((Y == 1) & (Y_pred == 0), 1,
                 np.where((Y == 0) & (Y_pred == 1), 5, 0))

    # 4. Return average cost
        
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    
    return [cost_array.mean(), cost_array]

print(cost(model,0.5,X_test,Y_test))

[0.41726475279106856, array([0, 0, 0, ..., 0, 0, 0])]


In [45]:

# Part 5
import numpy as np

threshold_grid = np.linspace(0, 1, 101)

optimal_threshold = None
best_cost = float("inf")

for thr in threshold_grid:
    c = cost(model, thr, X_test, Y_test)[0]
    if c < best_cost:
        best_cost = c
        best_threshold = thr

print("Best threshold:", best_threshold)
print("Best cost on validation set:", best_cost)

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.
cost_at_optimal_threshold = best_cost

Best threshold: 0.8
Best cost on validation set: 0.2727272727272727


In [52]:
#we have that the cost is our empirical mean (mu^), we want to bound the probability of the difference between mu^ and E(c_i) (expected cost).
# by hoeffdings: P(abs(mu^-E(c_i))>=t)<=2e^(-2nt/(b-a)^2) where a=0 and b=5 in this case, since each 0<=c_i<=5
# we want P(abs(mu^-E(c_i))>=t)<=0.01 (99% confidence). doing some algebra we get:
n=len(Y_val)
t=np.sqrt(25*np.log(1/0.005)/(2*n))
cost_at_optimal_threshold_valid = cost(model,thr,X_val,Y_val)[0]
cost_interval_valid = (cost_at_optimal_threshold_valid-t,cost_at_optimal_threshold_valid+t)
print(cost_at_optimal_threshold_valid)
print(cost_interval_valid)
assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

0.5715139918679741
(0.48251831524798433, 0.660509668487964)


In [50]:

# Part 7
import numpy as np
from math import log

def b_function(u, c):
    # b(u,c) = (1 + c*u)*log(1 + c*u) - c*u
    return (1 + c*u) * np.log(1 + c*u) - c*u

def bennett_Q(t, n, sigma, alpha=0.01, b_max=5):
    """
    Two-sided 'exceedance' function for Bennett:
      Q(t) = P(|X_bar - mu| >= t)
    We'll find t s.t. Q(t) = alpha.
    """
    u = t / (sigma**2 + 1e-15)  # avoid /0
    c = b_max / (sigma + 1e-15)
    exponent = - n * (sigma**2) * b_function(u, c)
    return 2 * np.exp(exponent)  # two-sided => multiply by 2

def find_t_bennett(n, sigma, alpha=0.01, b_max=5):
    """Search for t >= 0 s.t. bennett_Q(t)=alpha."""
    # We'll do a simple binary search in [0, b_max].
    # The maximum cost is 5, so the mean can't exceed 5, but we give some margin:
    left, right = 0.0, 5.0
    for _ in range(50):  # 50 iterations is plenty
        mid = 0.5 * (left + right)
        if bennett_Q(mid, n, sigma, alpha=alpha, b_max=b_max) > alpha:
            left = mid
        else:
            right = mid
    return 0.5 * (left + right)

n_val = len(Y_val)

hat_mu = cost_at_optimal_threshold_valid
hat_sigma = cost(model,thr,X_val,Y_val)[1].std(ddof=1)
alpha = 0.01  # 99% CI
t_star = find_t_bennett(n_val, hat_sigma, alpha=alpha, b_max=5)
variance_of_C = hat_sigma
interval_of_C = (hat_mu-t_star,hat_mu+t_star)
print(interval_of_C)
assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)


(0.5697494997563364, 0.5732784839796119)
